# Setup distance matrix calculation

In [1]:
import sys
sys.path.insert(0, "../src")

In [2]:
import numpy as np
import pandas as pd
import xarray
from scipy.spatial.distance import cdist
from geopy.distance import geodesic
from sklearn.metrics.pairwise import haversine_distances

import data_utils as utils
import geostats as gs

In [3]:
DS_sif = xarray.open_dataset("../data/exp_pro/OCO2_Lite_SIF740.nc")
DS_xco2 = xarray.open_dataset("../data/exp_pro/OCO2_Lite_XCO2_land.nc")

In [4]:
df_sif_monthly = (
    utils.regrid(DS_sif, res=5)
    .groupby(["lon", "lat"])
    .resample("1MS")
    .mean()
    .drop(columns=["lon", "lat"])
    .reset_index()
)
df_xco2_monthly = (
    utils.regrid(DS_xco2, res=5)
    .groupby(["lon", "lat"])
    .resample("1MS")
    .mean()
    .drop(columns=["lon", "lat"])
    .reset_index()
)

# Merge dataframes
DS_grid = (
    pd.merge(df_sif_monthly, df_xco2_monthly, on=["lon", "lat", "time"], how="outer")
    .set_index(["lon", "lat", "time"])
    .to_xarray()
)
DS_grid

<xarray.Dataset>
Dimensions:  (lat: 35, lon: 72, time: 71)
Coordinates:
  * lon      (lon) float64 -177.5 -172.5 -167.5 -162.5 ... 167.5 172.5 177.5
  * lat      (lat) float64 -87.5 -82.5 -77.5 -72.5 -67.5 ... 67.5 72.5 77.5 82.5
  * time     (time) datetime64[ns] 2014-09-01 2014-10-01 ... 2020-07-01
Data variables:
    sif      (lon, lat, time) float32 nan nan 0.013318249 ... nan nan nan
    xco2     (lon, lat, time) float32 nan nan nan nan nan ... nan nan nan nan

In [28]:
DS_grid.sif.sel(time="2020-03-01").to_dataframe().reset_index().values[:,-1]

array([nan, nan, nan, ..., nan, nan, nan], dtype=object)

In [41]:
class Field:
    """
    Stores data values and coordinates for a single process.
    """

    def __init__(self, da, timestamp):
        df = da.sel(time=timestamp).to_dataframe().reset_index()
        self.timestamp = datetime.strptime(timestamp, "%Y-%m-%d")
        self.coords = df[["lat", "lon"]].values
        self.values = df.values[:, -1]
        
        self.lag = (self.timestamp - relativedelta(months=3)).strftime("%Y-%m-%d")
        
        
Z1 = Field(DS_grid.xco2, "2020-03-01")
Z2 = Field(DS_grid.sif, "2020-03-01")

Z1.lag

'2019-12-01'

In [74]:
geodesic((DS_grid.lat.values[0], DS_grid.lon.values[0]), (DS_grid.lat.values[1], DS_grid.lon.values[1])).miles

347.9753814124189

In [5]:
a = np.arange(2)
b = np.arange(2)

def expand_grid(*args):
    """
    Returns an array of all combinations of elements in the supplied vectors.
    """
    return np.array(np.meshgrid(*args)).T.reshape(-1,len(args))

def distance_matrix(X1, X2=None, units="km"):
    """
    Computes the geodesic distance among all pairs of points given two sets of coordinates.
    Wrapper for scipy.spatial.distance.cdist using geopy.distance.geodesic as a the metric.
    """
    if X2 is not None:
        return cdist(X1, X2, lambda s_i, s_j: getattr(geodesic(s_i, s_j), units))
    else:
        return cdist(X1, X1, lambda s_i, s_j: getattr(geodesic(s_i, s_j), units))
    
def distance_matrix_fast(X1, X2):
    """
    Computes the Haversine (or Great Circle) distance among all pairs of points (in kilometers).
    """
    EARTH_RADIUS = 6371 # radius in kilometers
    X1_r = np.radians(X1)
    X2_r = np.radians(X2)
    return haversine_distances(X1_r, X2_r) * EARTH_RADIUS

In [6]:
coords = expand_grid(DS_grid.lat.values[:2], DS_grid.lon.values[:2])
distance_matrix(coords, units="miles")

array([[  0.        ,  15.13178313, 346.98826934, 347.97538141],
       [ 15.13178313,   0.        , 347.97538141, 346.98826934],
       [346.98826934, 347.97538141,   0.        ,  45.27809565],
       [347.97538141, 346.98826934,  45.27809565,   0.        ]])

In [81]:
geodesic((-87.5, -177.5), (-87.5, -172.5)).km

396 µs ± 5.87 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [73]:
coords = expand_grid(DS_grid.lat.values, DS_grid.lon.values)
distance_matrix(coords)

array([[    0.        ,    24.3522444 ,    48.65822076, ...,
        18894.21642581, 18890.234938  , 18887.84064812],
       [   24.3522444 ,     0.        ,    24.3522444 , ...,
        18899.77158275, 18894.21642581, 18890.234938  ],
       [   48.65822076,    24.3522444 ,     0.        , ...,
        18906.88139558, 18899.77158275, 18894.21642581],
       ...,
       [18894.21642581, 18899.77158275, 18906.88139558, ...,
            0.        ,    72.86803157,   145.59970868],
       [18890.234938  , 18894.21642581, 18899.77158275, ...,
           72.86803157,     0.        ,    72.86803157],
       [18887.84064812, 18890.234938  , 18894.21642581, ...,
          145.59970868,    72.86803157,     0.        ]])

In [107]:
distance_matrix_fast(coords, coords)

array([[    0.        ,    24.24359308,    48.44112457, ...,
        18910.27792814, 18906.31551111, 18903.93269494],
       [   24.24359308,     0.        ,    24.24359308, ...,
        18915.80649549, 18910.27792814, 18906.31551111],
       [   48.44112457,    24.24359308,     0.        , ...,
        18922.88230969, 18915.80649549, 18910.27792814],
       ...,
       [18910.27792814, 18915.80649549, 18922.88230969, ...,
            0.        ,    72.54661905,   144.95748508],
       [18906.31551111, 18910.27792814, 18915.80649549, ...,
           72.54661905,     0.        ,    72.54661905],
       [18903.93269494, 18906.31551111, 18910.27792814, ...,
          144.95748508,    72.54661905,     0.        ]])